# Set references for v8 tuning and export to ONNX


In [1]:
from core import crossval_table, fit_table
from Gaugi import mkdir_p
from sklearn.model_selection import StratifiedKFold, KFold
from multiprocessing import Process
from time import sleep
import saphyra
import numpy as np
import pandas as pd
import collections
import os
import json
import matplotlib
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, model_from_json


from pprint import pprint
from copy import deepcopy, copy
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence

Applying ATLAS style settings...

Applying ATLAS style settings...
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...


In [2]:
output_path = 'output/fitting'
mkdir_p(output_path)

In [3]:
def create_op_dict(op, decoration='reference'):
    
    d = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              
              # op
              'pd_ref'    : decoration+"/"+op+"/pd_ref#0",
              'fa_ref'    : decoration+"/"+op+"/fa_ref#0",
              'sp_ref'    : decoration+"/"+op+"/sp_ref",
              'pd_val'    : decoration+"/"+op+"/pd_val#0",
              'fa_val'    : decoration+"/"+op+"/fa_val#0",
              'sp_val'    : decoration+"/"+op+"/sp_val",
              'pd_op'     : decoration+"/"+op+"/pd_op#0",
              'fa_op'     : decoration+"/"+op+"/fa_op#0",
              'sp_op'     : decoration+"/"+op+"/sp_op",

              # Counts
              'pd_ref_passed'    : decoration+"/"+op+"/pd_ref#1",
              'fa_ref_passed'    : decoration+"/"+op+"/fa_ref#1",
              'pd_ref_total'     : decoration+"/"+op+"/pd_ref#2",
              'fa_ref_total'     : decoration+"/"+op+"/fa_ref#2",
              'pd_val_passed'    : decoration+"/"+op+"/pd_val#1",
              'fa_val_passed'    : decoration+"/"+op+"/fa_val#1",
              'pd_val_total'     : decoration+"/"+op+"/pd_val#2",
              'fa_val_total'     : decoration+"/"+op+"/fa_val#2",
              'pd_op_passed'     : decoration+"/"+op+"/pd_op#1",
              'fa_op_passed'     : decoration+"/"+op+"/fa_op#1",
              'pd_op_total'      : decoration+"/"+op+"/pd_op#2",
              'fa_op_total'      : decoration+"/"+op+"/fa_op#2",
    })
    return d


op_names = ['tight', 'medium', 'loose', 'vloose']

tuned_info = collections.OrderedDict({})
for op in op_names:
    tuned_info[op] = create_op_dict(op, "reference")

In [4]:
etbins = [15, 20, 30, 40, 50,100, 1000000]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]

## 1) Reading all tunings:


In [5]:
cv  = crossval_table( tuned_info, etbins = etbins , etabins = etabins )

In [10]:
best_inits = pd.read_csv('output/crossval/best_inits_v16_30bins.csv', index_col=[0])

In [11]:
best_inits = best_inits.loc[(best_inits.model_idx==0)] # 5 neurons for all phase spaces
best_sorts = cv.filter_sorts( best_inits , 'max_sp_op')

In [12]:
basepath = '/home/jodafons/public/cern_data/new_files/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'
datapath = basepath+'/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.h5'
paths = [ [datapath.format(ET=et_bin, ETA=eta_bin) for eta_bin in range(5)] for et_bin in range(5)]

In [13]:
# calculate all pd/fa from reference file
ref_path = '/home/jodafons/public/cern_data/new_files/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins/'
ref_path += 'new_references/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.ref.npz'
ref_paths = [[ ref_path.format(ET=et,ETA=eta) for eta in range(5)] for et in range(5)]
ref_values = [[ {} for eta in range(5)] for et in range(5)]

from saphyra.core import ReferenceReader
for et_bin in range(5):
    for eta_bin in range(5):
        for op_name in op_names:
            refObj = ReferenceReader().load(ref_paths[et_bin][eta_bin])
            _pd = refObj.getSgnPassed(op_name)/refObj.getSgnTotal(op_name)
            _fa = refObj.getBkgPassed(op_name)/refObj.getBkgTotal(op_name)
            ref_values[et_bin][eta_bin][op_name] = {'pd':_pd, 'fa':_fa, 'pd_epsilon':0.0}

In [14]:
def getPatterns( path, etbin, etabin):

    pidname = 'el_lhmedium'
    from kepler.pandas import load_hdf
    import numpy as np
    df = load_hdf(path)
    df = df.loc[ ((df[pidname]==True) & (df.target==1.0)) | ((df.target==0) & (df['el_lhvloose']==False) ) ]
    df = df.loc[ (df['trig_L2_cl_et']/1000  >= etbin[0]) & (df['trig_L2_cl_et']/1000  < etbin[1])]
    df = df.loc[ (abs(df['trig_L2_cl_eta'])  >= etabin[0]) & (abs(df['trig_L2_cl_eta'])  < etabin[1])]

      

    def reshape_to_vortex( input_data):
    
        # NOTE: Do not change this if you dont know what are you doing
        frame =     [ [72,73,74,75,76,77,78,79,80,81],
                      [71,42,43,44,45,46,47,48,49,82],
                      [70,41,20,21,22,23,24,25,50,83],
                      [69,40,19,6 ,7 ,8 ,9 ,26,51,84],
                      [68,39,18,5 ,0 ,1 ,10,27,52,85],
                      [67,38,17,4 ,3 ,2 ,11,28,53,86],
                      [66,37,16,15,14,13,12,29,54,87],
                      [65,36,35,34,33,32,31,30,55,88],
                      [64,63,62,61,60,59,58,57,56,89],
                      [99,98,97,96,95,94,93,92,91,90],
                    ]
        from copy import deepcopy
        zeros_to_complete = np.zeros((input_data.shape[0],100-input_data.shape[1]))
        data = deepcopy(np.hstack([input_data, zeros_to_complete]))
        d = deepcopy(data.reshape( 1,10,10,data.shape[0] ))
        data=data.T
        for i in range(10):
            for j in range(10):
                d[0][i][j][::] = data[ frame[i][j] ][::]
        d=d.T
        return d
   
    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    
    col_names= ['trig_L2_cl_ring_%d'%i for i in range(100)]
    rings = df[col_names].values.astype(np.float32)
    data_rings = norm1(rings)
    data_rings = reshape_to_vortex(data_rings)
    target = df['target'].values.astype(np.int16)
    avgmu = df.avgmu.values
  
    return [data_rings], target, avgmu

In [15]:
paths.append(paths[4])
ref_values.append(ref_values[4])

## Linear correction:

In [16]:
kf = StratifiedKFold(n_splits=10, random_state=512, shuffle=True)

In [17]:
cv_fit = fit_table( etbins, etabins, kf )

**NOTE**: This step will take something like 5 min to be ready

In [18]:
best_sorts_refit = cv_fit.fill( best_sorts , getPatterns, paths, ref_values, output_path=output_path ,
                                min_avgmu=16, max_avgmu=60)

Filling... : 100%|████████████████████| 30/30 [12:00<00:00, 24.00s/it]


In [19]:
best_sorts_refit.to_csv('output/fitting/best_sorts_v16.csv')

In [20]:
for op in op_names:
    cv_fit.dump_beamer_table( best_sorts_refit.loc[best_sorts_refit.op_name == op] ,                  
                              op+' Fitting (v16)', 'fitting_v16_'+op)

2022-03-24 19:43:08,351 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file fitting_v16_tight.pdf latex code...
2022-03-24 19:43:11,896 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file fitting_v16_medium.pdf latex code...
2022-03-24 19:43:14,873 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file fitting_v16_loose.pdf latex code...
2022-03-24 19:43:17,886 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file fitting_v16_vloose.pdf latex code...


In [21]:
!mv *.pdf output/fitting
!mv *.tex output/fitting

## Export all tunings:

In [22]:
model_name_format = 'data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.model_v16.electron{op}.et%d_eta%d'
config_name_format = 'ElectronRinger{op}TriggerConfig.conf'
op_capnames = ['Tight', 'Medium', 'Loose', 'VeryLoose']
for idx, op in enumerate( ['tight','medium','loose','vloose'] ):
    cv_fit.export(best_sorts_refit.loc[best_sorts_refit.op_name==op], 
                  model_name_format.format(op=op_capnames[idx]), 
                  config_name_format.format(op=op_capnames[idx]), 
                  op, 
                  to_onnx     = True,
                  remove_last = True,
                  min_avgmu   = 16,
                  max_avgmu   = 100)

2022-03-24 19:44:33,780 | Py.fit_table                            INFO Export all tuning configuration to ElectronRingerTightTriggerConfig.conf.
2022-03-24 19:45:45,158 | Py.fit_table                            INFO Export all tuning configuration to ElectronRingerMediumTriggerConfig.conf.
2022-03-24 19:46:55,203 | Py.fit_table                            INFO Export all tuning configuration to ElectronRingerLooseTriggerConfig.conf.
2022-03-24 19:48:05,247 | Py.fit_table                            INFO Export all tuning configuration to ElectronRingerVeryLooseTriggerConfig.conf.


tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -

In [23]:
!mv models output/fitting
!mv *.conf output/fitting
!mv *.tex output/fitting
!mv *.pdf output/fitting

mv: cannot move 'models' to 'output/fitting/models': Directory not empty
mv: cannot stat '*.tex': No such file or directory
mv: cannot stat '*.pdf': No such file or directory
